## -- 1. Criar uma Tabela Delta a partir de uma tabela existente (trips_raw)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mc_labs.trips_delta
USING DELTA
AS SELECT
 tpep_pickup_datetime,
 tpep_dropoff_datetime,
 trip_distance,
 fare_amount,
 pickup_zip,
 dropoff_zip
FROM mc_labs.trips_raw;

-- Verificar se a tabela foi criada e contém dados
SELECT * FROM mc_labs.trips_delta LIMIT 10;

## -- 2. Inserir Dados em uma Tabela Delta
-- Podemos inserir mais dados, ou até mesmo dados de outra fonte
-- Para simplificar, vamos inserir alguns dados de exemplo.

In [0]:
%sql
INSERT INTO mc_labs.trips_delta VALUES
('2023-01-01 10:00:00', '2023-01-01 10:15:00', 2.5, 15.00, 10001, 10002),
('2023-01-01 11:00:00', '2023-01-01 11:30:00', 5.0, 25.50, 10003, 10004);

-- Verificar os novos dados
SELECT * FROM mc_labs.trips_delta WHERE pickup_zip IN (10001, 10003);


## -- 3. Atualizar Dados (UPDATE)
-- Vamos atualizar o fare_amount para viagens com um determinado pickup_zip

In [0]:
%sql
UPDATE mc_labs.trips_delta
SET fare_amount = fare_amount * 1.10 -- Aumentar em 10%
WHERE pickup_zip = 10001;

-- Verificar a atualização
SELECT * FROM mc_labs.trips_delta WHERE pickup_zip = 10001;


## -- 4. Excluir Dados (DELETE)
-- Vamos excluir as viagens com um determinado dropoff_zip

In [0]:
%sql
DELETE FROM mc_labs.trips_delta
WHERE dropoff_zip = 10004;

-- Verificar a exclusão
SELECT * FROM mc_labs.trips_delta WHERE dropoff_zip = 10004; -- Deve retornar 0 linhas


## -- 5. Upsert (MERGE INTO) - Uma das características mais poderosas do Delta Lake
-- MERGE INTO permite inserir novas linhas, atualizar linhas existentes ou ambas,
-- com base em uma condição de correspondência.
-- Cenário: Temos novos dados de viagens. Se a viagem já existe (mesmo pickup/dropoff datetime e zips),
-- queremos atualizar o fare_amount. Caso contrário, queremos inserir a nova viagem.


In [0]:
%sql
-- Primeiro, vamos criar uma tabela temporária com os dados de origem para o MERGE
CREATE OR REPLACE TEMPORARY VIEW new_trip_data AS
SELECT * FROM VALUES
('2023-01-01 10:00:00', '2023-01-01 10:15:00', 2.5, 16.50, 10001, 10002), -- Viagem existente, fare_amount atualizado
('2023-01-02 09:00:00', '2023-01-02 09:20:00', 3.0, 18.00, 10005, 10006)  -- Nova viagem
AS new_data(tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, fare_amount, pickup_zip, dropoff_zip);

-- Executar o MERGE INTO
MERGE INTO mc_labs.trips_delta AS target
USING new_trip_data AS source
ON target.tpep_pickup_datetime = source.tpep_pickup_datetime
AND target.pickup_zip = source.pickup_zip
AND target.dropoff_zip = source.dropoff_zip
WHEN MATCHED THEN
 UPDATE SET
     target.fare_amount = source.fare_amount,
     target.trip_distance = source.trip_distance
WHEN NOT MATCHED THEN
 INSERT (tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, fare_amount, pickup_zip, dropoff_zip)
 VALUES (source.tpep_pickup_datetime, source.tpep_dropoff_datetime, source.trip_distance, source.fare_amount, source.pickup_zip, source.dropoff_zip);

-- Verificar os resultados do MERGE
SELECT * FROM mc_labs.trips_delta WHERE pickup_zip IN (10001, 10005);


## -- 6. Time Travel (Viagem no Tempo)
-- O Delta Lake mantém um histórico de todas as operações.
-- Podemos ver o histórico da tabela:
DESCRIBE HISTORY mc_labs.trips_delta;


In [0]:
%sql
-- Consultar uma versão anterior da tabela (por exemplo, a versão 0, que é o estado inicial)
-- A versão 0 deve ser o estado da tabela antes de qualquer INSERT, UPDATE, DELETE ou MERGE.
SELECT * FROM mc_labs.trips_delta VERSION AS OF 0 LIMIT 10;

-- Consultar uma versão anterior por timestamp (substitua pelo timestamp real de uma operação anterior)
-- SELECT * FROM mc_labs.trips_delta TIMESTAMP AS OF '2023-10-25 22:00:00' LIMIT 10;
-- (Você precisaria de um timestamp de uma operação anterior real para testar isso)


## -- 7. Schema Enforcement (Exemplo de falha)
-- Tentar inserir dados com um tipo de dado incorreto deve falhar

In [0]:
%sql
-- INSERT INTO mc_labs.trips_delta VALUES
--('2023-01-01 10:00:00', '2023-01-01 10:15:00', 'INVALID_DISTANCE', 15.00, 10001, 10002);
-- Descomente e execute para ver o erro de schema enforcement.

## -- 8. Schema Evolution (Adicionar uma nova coluna)
-- Podemos adicionar uma nova coluna à tabela Delta

In [0]:
%sql
ALTER TABLE mc_labs.trips_delta ADD COLUMN payment_type STRING;

-- Verificar o esquema atualizado
DESCRIBE mc_labs.trips_delta;

In [0]:
%sql
-- Inserir dados com a nova coluna (ou atualizar)
INSERT INTO mc_labs.trips_delta VALUES
('2023-01-03 12:00:00', '2023-01-03 12:10:00', 1.8, 12.00, 10007, 10008, 'Credit Card');

SELECT * FROM mc_labs.trips_delta WHERE pickup_zip = 10007;

In [0]:
%sql
DESCRIBE HISTORY mc_labs.trips_delta